In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# Import necessary modules from LangChain
from langchain.chat_models import ChatOpenAI
from talk_openai import MyOpenAI
from langchain.schema import HumanMessage
import os
from utils import *
import re

In [55]:
#gathered information
#container name and app name cannot have underscores
options = {
    "docker_image": "neel26d/iris_model_inference:latest",
    "docker_container_name": "iris_model-inference",
    "model_name": 'iris-model',
    "model_port": 8000,
    "service_port": 80,
    "number_of_replicas": 1,
    'deployment_template': file_reader('/Users/neel/Developer/deploy_wizard/templates/k8s/deployment.yaml'),
    "service_template": file_reader('templates/k8s/service.yaml'),
    'where_is_cluster': 'minikube',}


In [56]:
k8s_gen_llm = MyOpenAI(model="gpt-4o-mini")
k8_critic = MyOpenAI(model="gpt-4o-mini")

In [57]:
def generate_kubernetes_yaml(options,feedback=None):
    # Generate the Kubernetes YAML configuration
    if feedback is None:
        messages = [
            {"role": "system", "content": "You are an expert in Kubernetes and Docker. You will be given a set of options to create a Kubernetes YAML configuration for a Docker container. The options will include the Docker image, container name, model name, model port, and service port. You will also be given feedback from a critic. Use this feedback to improve the generated YAML configuration."},
            {"role": "user", 
            "content": f'''You are an expert Kubernetes YAML generator.
            Generate two YAML files:
            1. A Kubernetes Deployment manifest.
            2. A Kubernetes Service manifest.

            **Deployment Requirements:**
            - Application Name: {options['model_name']}
            - Container Image: {options['docker_image']}
            - Container Port: {options['model_port']}
            - Number of Replicas: {options['number_of_replicas']}
            - Labels: app: {options['model_name']},
            - The container must expose port {options['model_port']}.
            - Template to use as reference: {options['deployment_template']}

            **Service Requirements:**
            - Service Name: {options['model_name']}-service
            - Service Type: LoadBalancer
            - Expose Port: {options['service_port']} externally.
            - Target Port: {options['model_port']} internally.
            - Selector should match the Deployment labels.
            - Template to use as reference: {options['service_template']}

            Format the output with proper YAML syntax.
            Provide each manifest separately under clear headings like:
            `--- Deployment ---` and `--- Service ---`.
            The cluster is in {options['where_is_cluster']}.'''},
        ]
    else:
        messages = feedback
    response = k8s_gen_llm.invoke(messages)
    return response


def evaluate_kubernetes_with_llm(kubernetes_code):
    """Use an LLM to evaluate the generated Kubernetes YAML and provide feedback."""
    messages = [
        {"role": "system", "content": "You are a Kubernetes expert and YAML code reviewer specializing in deployments and services."},
        {"role": "user", "content": f"""
        You are reviewing Kubernetes YAML manifests for deploying and exposing an application.

        ### Task:
        - Analyze the YAMLs for correctness, best practices, and completeness.
        - Validate that the Deployment and Service fields are correctly specified (apiVersion, kind, metadata, spec).
        - Ensure that labels and selectors match properly between Deployment and Service.
        - Check that the container port and Service targetPort are correctly aligned.
        - Verify that the Service type (LoadBalancer, NodePort, etc.) is appropriate.
        - Confirm the YAML structure is clean, readable, and properly organized.
        - Ensure no unnecessary fields or missing mandatory fields are present.
        - If the YAMLs are perfect, respond with 'No changes needed.'
        - If changes are required, provide a detailed improvement plan.
        - Do not introduce new labels, ports, or fields unless absolutely necessary based on the provided YAML.
        - Do not ask it to add any stuff for which information is not provided.

        ### YAMLs to review:
        ```
        {kubernetes_code}
        ```

        ### Expected Output:
        - Either 'No changes needed.' OR a structured list of suggestions for improvement.
        """}
    ]

    return k8_critic.invoke(messages)

In [58]:
def actor_critic_k8s(params):
    feedback = None  # No feedback initially
    # Step 1: Actor generates Terraform script
    kubernetes_code = generate_kubernetes_yaml(params, feedback)
    for _ in range(2):  # Max iterations: 2
        # Step 2: Critic evaluates the Terraform script
        critic_feedback = evaluate_kubernetes_with_llm(kubernetes_code)

        if "no changes needed" in critic_feedback.lower():
            print("✅ No changes needed. Returning final Kubernetes YAML.")
            return kubernetes_code  # Early exit if script is sufficient

        # Step 3: Actor refines Terraform script using critic's feedback
        feedback = [
            {"role": "assistant", "content": kubernetes_code},
            {"role": "user", "content": f"Revise based on this feedback:\n{critic_feedback}"}
        ]

        kubernetes_code = generate_kubernetes_yaml(params, feedback)


    print("🔄 Max iterations reached. Returning final Kubernetes files.")
    return kubernetes_code


In [59]:
k8s_files = actor_critic_k8s(options)

🔄 Max iterations reached. Returning final Kubernetes files.


In [60]:
print(k8s_files)

Here’s the revised Kubernetes YAML configuration for the Deployment and Service, incorporating your feedback and suggestions for clarity, completeness, and best practices:

```yaml
# Deployment for the iris-model application
apiVersion: apps/v1
kind: Deployment
metadata:
  name: iris-model
  labels:
    app: iris-model
spec:
  replicas: 3  # Increased for high availability; adjust based on expected load
  selector:
    matchLabels:
      app: iris-model
  template:
    metadata:
      labels:
        app: iris-model
    spec:
      containers:
      - name: irismodel-inference
        image: neel26d/iris_model_inference:latest
        ports:
        - containerPort: 8000
        resources:  # Resource requests and limits for better resource management
          requests:
            memory: "64Mi"  # Adjust based on actual usage
            cpu: "250m"     # Adjust based on actual usage
          limits:
            memory: "128Mi" # Adjust based on actual usage
            cpu: "500m"

In [61]:
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
import os

# Define the saving tool
@tool
def save_file_tool(file_path: str, content: str) -> str:
    """Save content to a file at the given path."""
    os.makedirs(os.path.dirname(file_path), exist_ok=True)
    with open(file_path, 'w') as f:
        f.write(content)
    return f"Saved to {file_path}"

# Define the function
def extract_and_save_k8s_files(k8s_text: str, save_dir: str = './k8s_files'):
    llm = ChatOpenAI(model="gpt-4o-mini")

    tools = [save_file_tool]

    agent = initialize_agent(
        tools=tools,
        llm=llm,
        agent=AgentType.OPENAI_FUNCTIONS,
        verbose=True,
    )

    response = agent.invoke(
        f"""
        1. Extract all Kubernetes YAML code blocks from the given text.
        2. For each extracted YAML:
            - If it is a Deployment, save it as '{save_dir}/deployment.yaml'.
            - If it is a Service, save it as '{save_dir}/service.yaml'.
            - Otherwise, save it as '{save_dir}/other.yaml'.
        3. Use the `save_file_tool` tool to save the content.
        4. Do not output anything else except confirming tool usage.

        ### Text:
        {k8s_text}
        """
    )

    return response

In [62]:
response= extract_and_save_k8s_files(k8s_files)



> Entering new AgentExecutor chain...

Invoking: `save_file_tool` with `{'file_path': './k8s_files/deployment.yaml', 'content': '# Deployment for the iris-model application\napiVersion: apps/v1\nkind: Deployment\nmetadata:\n  name: iris-model\n  labels:\n    app: iris-model\nspec:\n  replicas: 3  # Increased for high availability; adjust based on expected load\n  selector:\n    matchLabels:\n      app: iris-model\n  template:\n    metadata:\n      labels:\n        app: iris-model\n    spec:\n      containers:\n      - name: irismodel-inference\n        image: neel26d/iris_model_inference:latest\n        ports:\n        - containerPort: 8000\n        resources:  # Resource requests and limits for better resource management\n          requests:\n            memory: "64Mi"  # Adjust based on actual usage\n            cpu: "250m"     # Adjust based on actual usage\n          limits:\n            memory: "128Mi" # Adjust based on actual usage\n            cpu: "500m"     # Adjust based on

In [46]:
llm_validator = MyOpenAI(model="gpt-4o")
def k8s_deployment_validator(saved_files_dir: str = './k8s_files'):
    files = os.listdir(saved_files_dir)
    contents = ''
    for file in files:
        with open(os.path.join(saved_files_dir, file), 'r') as f:
            contents += file
            contents += f.read()
    msg = f'''
    The user has deployed a buch of kubernetes stuff \n {contents} on {options['where_is_cluster']}. \n
    Please help the user validate if everything is working as expected. And provide instruction on how to test the service from outside the cluster. \n'''
    response = llm_validator.invoke(msg)
    print(response)
    while True:
        msg = input("Enter your response, if everything is working say gg: ")
        if msg == 'gg':
            break
        print("User: ",msg)
        response = llm_validator.invoke(msg)
        print(response)

In [63]:
k8s_deployment_validator()

To validate that your Kubernetes deployment and service are working as expected on Minikube, and to test the service from outside the cluster, follow these steps:

### Step 1: Verify Deployment and Service

1. **Check the Deployment:**
   - Ensure that the deployment is up and running with the desired number of replicas:
     ```bash
     kubectl get deployments
     ```
   - You should see the `iris-model` deployment listed with 3 replicas.

2. **Check the Pods:**
   - Verify that the pods are running:
     ```bash
     kubectl get pods
     ```
   - Ensure that all pods associated with the `iris-model` deployment are in the `Running` state.

3. **Check the Service:**
   - Verify that the service is created and has an external IP assigned:
     ```bash
     kubectl get services
     ```
   - You should see the `iris-model-service` with a `LoadBalancer` type. The `EXTERNAL-IP` column might show `<pending>` initially, but with Minikube, you will use `minikube tunnel` to expose the servi